In [1]:
import sys
sys.path.append('..')

import numpy as np
import torch
from torch import nn
from torch.autograd import Variable
from torchvision.datasets import CIFAR10

In [2]:
def conv_relu(in_channel,out_channel,kernel,strdie=1,padding=0):
    layer=nn.Sequential(
        nn.Conv2d(in_channel,out_channel,kernel,strdie,padding),
        nn.BatchNorm2d(out_channel,eps=1e-3),
        nn.ReLU(True)
    )
    return layer

In [3]:
class inception(nn.Module):
    def __init__(self,in_channel,out1_1,out2_1,out2_3,out3_1,out3_5,out4_1):
        super(inception,self).__init__()
        # first line
        self.branch1x1=conv_relu(in_channel,out1_1,1)
        
        # second line
        self.branch3x3=nn.Sequential(
            conv_relu(in_channel,out2_1,1),
            conv_relu(out2_1,out2_3,3,padding=1)
        )
        
        # thrid line
        self.branch5x5=nn.Sequential(
            conv_relu(in_channel,out3_1,1),
            conv_relu(out3_1,out3_5,5,padding=2)
        )
        # four line
        self.branch_pool=nn.Sequential(
            nn.MaxPool2d(3,stride=1,padding=1),
            conv_relu(in_channel,out4_1,1)
        )
    
    def forward(self,x):
        f1=self.branch1x1(x)
        f2=self.branch3x3(x)
        f3=self.branch5x5(x)
        f4=self.branch_pool(x)
        output=torch.cat((f1,f2,f3,f4),dim=1)
        return output

In [4]:
test_net=inception(3,64,48,64,64,96,32)
test_x=Variable(torch.zeros(1,3,96,96))
print('input shape: {} x {} x {}'.format(test_x.shape[1], test_x.shape[2], test_x.shape[3]))
test_y = test_net(test_x)
print('output shape: {} x {} x {}'.format(test_y.shape[1], test_y.shape[2], test_y.shape[3]))

input shape: 3 x 96 x 96
output shape: 256 x 96 x 96


In [5]:
class googlenet(nn.Module):
    def __init__(self,in_channel,num_classes,verbose=False):
        super(googlenet,self).__init__()
        self.verbose=verbose
        
        self.block1=nn.Sequential(
            conv_relu(in_channel,out_channel=64,kernel=7,strdie=2,padding=3),
            nn.MaxPool2d(3,2)
        )
        
        self.block2=nn.Sequential(
            conv_relu(64,64,kernel=1),
            conv_relu(64,192,kernel=3,padding=1),
            nn.MaxPool2d(32,2)
        )
        
        self.block3=nn.Sequential(
            inception(192,64,96,128,16,32,32),
            inception(256,128,128,192,32,96,64),
            nn.MaxPool2d(3,2)
        )
        
        self.block4=nn.Sequential(
            inception(480, 192, 96, 208, 16, 48, 64),
            inception(512, 160, 112, 224, 24, 64, 64),
            inception(512, 128, 128, 256, 24, 64, 64),
            inception(512, 112, 144, 288, 32, 64, 64),
            inception(528, 256, 160, 320, 32, 128, 128),
            nn.MaxPool2d(3, 2)
        )
        self.block5 = nn.Sequential(
            inception(832, 256, 160, 320, 32, 128, 128),
            inception(832, 384, 182, 384, 48, 128, 128),
            nn.AvgPool2d(2)
        )
        
        self.classifier = nn.Linear(1024, num_classes)
        
    def forward(self, x):
        x = self.block1(x)
        if self.verbose:
            print('block 1 output: {}'.format(x.shape))
        x = self.block2(x)
        if self.verbose:
            print('block 2 output: {}'.format(x.shape))
        x = self.block3(x)
        if self.verbose:
            print('block 3 output: {}'.format(x.shape))
        x = self.block4(x)
        if self.verbose:
            print('block 4 output: {}'.format(x.shape))
        x = self.block5(x)
        if self.verbose:
            print('block 5 output: {}'.format(x.shape))
        x = x.view(x.shape[0], -1)
        x = self.classifier(x)
        return x        

In [6]:
test_net = googlenet(3, 10, True)
test_x = Variable(torch.zeros(1, 3, 96, 96))
test_y = test_net(test_x)
print('output: {}'.format(test_y.shape))

block 1 output: torch.Size([1, 64, 23, 23])


RuntimeError: Given input size: (192x23x23). Calculated output size: (192x-3x-3). Output size is too small at c:\a\w\1\s\tmp_conda_3.6_091443\conda\conda-bld\pytorch_1544087948354\work\aten\src\thnn\generic/SpatialDilatedMaxPooling.c:51

In [7]:
from utils import train

def data_tf(x):
    x = x.resize((96, 96), 2) # 将图片放大到 96 x 96
    x = np.array(x, dtype='float32') / 255
    x = (x - 0.5) / 0.5 # 标准化，这个技巧之后会讲到
    x = x.transpose((2, 0, 1)) # 将 channel 放到第一维，只是 pytorch 要求的输入方式
    x = torch.from_numpy(x)
    return x
     
train_set = CIFAR10('./data', train=True, transform=data_tf)
train_data = torch.utils.data.DataLoader(train_set, batch_size=64, shuffle=True)
test_set = CIFAR10('./data', train=False, transform=data_tf)
test_data = torch.utils.data.DataLoader(test_set, batch_size=128, shuffle=False)

net = googlenet(3, 10)
optimizer = torch.optim.SGD(net.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

In [8]:
train(net, train_data, test_data, 20, optimizer, criterion)

RuntimeError: Given input size: (192x23x23). Calculated output size: (192x-3x-3). Output size is too small at c:\a\w\1\s\tmp_conda_3.6_091443\conda\conda-bld\pytorch_1544087948354\work\aten\src\thnn\generic/SpatialDilatedMaxPooling.c:51